In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import random

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
year='2013'
word2vec_size=50
cut_long_sentence=200
word2vec_model = KeyedVectors.load_word2vec_format('data/yelp'+year+'/yelp_'+year+'_'+str(word2vec_size)+'.vector')

2018-05-21 11:14:22,325 : INFO : loading projection weights from data/yelp2013/yelp_2013_50.vector
2018-05-21 11:14:24,646 : INFO : loaded (43530, 50) matrix from data/yelp2013/yelp_2013_50.vector


In [3]:
f_vocab=open('data/yelp'+year+'/yelp_'+year+'_'+str(word2vec_size)+'.vector')
vocab_set=[]
for line in f_vocab:
    vocab_set.append(line.strip().split(' ')[0])
vocab_set=set(vocab_set)
f_vocab.close()
print('vocabulary size =',len(vocab_set))

vocabulary size = 43531


In [4]:
num_classes=5

In [5]:
tf.reset_default_graph()

# Part I: word level attention
gru_hidden_size_low=128
attention_hidden_low=128

y=tf.placeholder(dtype=tf.int32,shape=[None])
y_onehot=tf.one_hot(y,num_classes)

X_sentence=tf.placeholder(dtype=tf.float32,shape=[None,None,None,word2vec_size]) # shape=(batch_size,doc_size,sen_size,vocab)
batch_size=tf.shape(X_sentence)[0]
document_size=tf.shape(X_sentence)[1]
sentence_size=tf.shape(X_sentence)[2]
X_sentence_r=tf.reshape(X_sentence,(batch_size*document_size,sentence_size,word2vec_size)) # shape=(batch_size*doc_size,sen_size,vocab)

sentence_length=tf.placeholder(dtype=tf.int32,shape=[None])  # it should have shape=(batch_size*document_size,)

cell_fw_low=tf.contrib.rnn.GRUCell(gru_hidden_size_low)
cell_bw_low=tf.contrib.rnn.GRUCell(gru_hidden_size_low)

word_attention_net_low=tf.layers.Dense(units=attention_hidden_low,name='word_attention_net_low')


with tf.variable_scope("word_level") as vs:
    # shape=(batch_size*doc_size,sen_size,gru_size)
    low_output=tf.nn.bidirectional_dynamic_rnn(cell_fw_low,cell_bw_low,
                                               dtype=tf.float32,
                                               inputs=X_sentence_r,
                                               sequence_length=sentence_length,
                                               time_major=False)

low_output=tf.concat(low_output[0],axis=2)  # shape=(batch_size*doc_size,sen_size,2*gru_size)

uw=tf.get_variable("uw", dtype=tf.float32, 
                   shape=(attention_hidden_low,1),
                   initializer=tf.random_normal_initializer(mean=0.0, stddev=1.0))

low_output_r=tf.reshape(low_output,(-1,2*gru_hidden_size_low))  # shape=(batch_size*doc_size*sen_size,2*gru_size)
low_output_r=word_attention_net_low(low_output_r)  # shape=(batch_size*doc_size*sen_size,attention_hidden_low)
score_low=tf.matmul(low_output_r,uw)  # shape=(batch_size*doc_size*sen_size,1)
score_low=tf.reshape(score_low,(batch_size*document_size,sentence_size))  # shape=(batch_size*doc_size,sen_size)
attention_low=tf.nn.softmax(score_low,dim=1)
sentence_vector=(tf.reshape(attention_low,(batch_size*document_size,sentence_size,1))*low_output)  # shape: same as low_output
sentence_vector=tf.reduce_sum(sentence_vector,axis=1)  # shape=(batch_size*doc_size,2*gru_size)
sentence_vector=tf.reshape(sentence_vector,(batch_size,document_size,2*gru_hidden_size_low))  # shape=(batch_size,doc_size,2*gru_size)


# Part II: sentence level attention
gru_hidden_size_high=128
attention_hidden_high=128

document_length=tf.placeholder(dtype=tf.int32,shape=[None])  # it should have shape=(batch_size,)

cell_fw_high=tf.contrib.rnn.GRUCell(gru_hidden_size_high)
cell_bw_high=tf.contrib.rnn.GRUCell(gru_hidden_size_high)

word_attention_net_high=tf.layers.Dense(units=attention_hidden_high,name='word_attention_net_high')

with tf.variable_scope("sentence_level") as vs:
    high_output=tf.nn.bidirectional_dynamic_rnn(cell_fw_high,cell_bw_high,
                                               dtype=tf.float32,
                                               inputs=sentence_vector,
                                               sequence_length=document_length,
                                               time_major=False)
high_output=tf.concat(high_output[0],axis=2)  # shape=(batch_size,doc_size,2*gru_size)

us=tf.get_variable("us", dtype=tf.float32, 
                   shape=(attention_hidden_high,1),
                   initializer=tf.random_normal_initializer(mean=0.0, stddev=1.0))

high_output_r=tf.reshape(high_output,(-1,2*gru_hidden_size_high))  # shape=(batch_size*doc_size,2*gru_size)
high_output_r=word_attention_net_high(high_output_r)  # shape=(batch_size*doc_size,attention)
score_high=tf.matmul(high_output_r,us)  # shape=(batch_size*doc_size,1)
score_high=tf.reshape(score_high,(batch_size,document_size))  # shape=(batch_size,doc_size)
attention_high=tf.nn.softmax(score_high,dim=1)  # shape=(batch_size,doc_size)
output=(tf.reshape(attention_high,(batch_size,document_size,1))*high_output)  # shape=(batch_size,doc_size,2*gru_size)
output=tf.reduce_sum(output,axis=1)  # shape=(batch_size,2*gru_size)


# Part III: predict
score=tf.layers.dense(output,units=num_classes,name='predict_net')

loss=tf.losses.softmax_cross_entropy(onehot_labels=y_onehot,logits=score)
predict=tf.cast(tf.argmax(score,axis=1),dtype=tf.int32)
accuracy=tf.reduce_mean(tf.cast(tf.equal(y,predict),dtype=tf.float32))

In [14]:
def make_input(doc):
    this_doc=[]
    sents=doc.split('\t')
    for s in sents:
        this_sentence=[]
        words=s.split(' ')
        for w in words:
            if w in vocab_set:
                this_sentence.append(w)
            else:
                this_sentence.append('<UNKNOWN>')
        this_doc.append(this_sentence)
        document_sizes=np.array([len(this_doc)])
        sentence_sizes=np.array([[len(t) for t in this_doc]])
        document_size=np.max(document_sizes)
        sentence_size=np.max(sentence_sizes)
        
        output=np.zeros((1,document_size,sentence_size,word2vec_size))
        
        for (id_s,s) in enumerate(this_doc):
            for (id_w,w) in enumerate(s):
                if id_w>=cut_long_sentence:  # cut too long sentences
                    break
                if w in word2vec_model:
                    output[0,id_s,id_w,:]=word2vec_model[w]
                else:
                    output[0,id_s,id_w,:]=np.ones(word2vec_size)
        
        
    return this_doc,document_sizes,sentence_sizes,output

In [35]:
f_dev=open('data/yelp'+year+'/yelp-'+year+'-dev-nonvocab.txt')  # randomly select some cases from dev set
cases=[]
for line in f_dev:
    cases.append(line.strip().split('\t\t'))
random.shuffle(cases)
cases=cases[:10]

In [62]:
saver=tf.train.Saver()
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    saver.restore(sess, "parameters/HAN.ckpt")
    cor=0
    for id,(label,sent) in enumerate(cases):
        this_doc,document_sizes,sentence_sizes,output=make_input(sent)
        label=np.array([int(label)-1])
        feed_dict={X_sentence:output,y:label,sentence_length:sentence_sizes.reshape(-1,),document_length:document_sizes}
        pre,al,ah=sess.run([predict,attention_low,attention_high],feed_dict=feed_dict)
        print('Case #'+str(id+1)+':')
        print('Original sentence:')
        print(', '.join(sent.split('\t')))
        print()
        print('Exact label: %d, predict label: %d' % (label[0]+1,pre[0]+1))
        print()
        print('Sentence attention:')
        for (id_s,s) in enumerate(this_doc):
            print('%.2f%%' % (ah[0,id_s]*100.0),' '.join(s))
        print()
        print('Word attention:')
        for (id_s,s) in enumerate(this_doc):
            print(' '.join([str(('%.2f%%' % (al[id_s,id_w]*100.0),w)) for (id_w,w) in enumerate(s)]))
        print()
        print()

INFO:tensorflow:Restoring parameters from parameters/HAN.ckpt


2018-05-21 13:21:05,054 : INFO : Restoring parameters from parameters/HAN.ckpt


Case #1:
Original sentence:
a hidden gem, service is awesome, prices are reasonable, food is very good, we came in with a large group of motorcycles, they greeted us and made room for us right away, i felt like they were sincerely glad to have our business and i will be back for sure

Exact label: 4, predict label: 5

Sentence attention:
0.00% a hidden gem
100.00% service is awesome
0.00% prices are reasonable
0.00% food is very good
0.00% we came in with a large group of motorcycles
0.00% they greeted us and made room for us right away
0.00% i felt like they were sincerely glad to have our business and i will be back for sure

Word attention:
('0.00%', 'a') ('67.61%', 'hidden') ('32.39%', 'gem')
('0.00%', 'service') ('0.00%', 'is') ('100.00%', 'awesome')
('0.00%', 'prices') ('0.00%', 'are') ('100.00%', 'reasonable')
('0.00%', 'food') ('0.00%', 'is') ('0.00%', 'very') ('100.00%', 'good')
('0.00%', 'we') ('100.00%', 'came') ('0.00%', 'in') ('0.00%', 'with') ('0.00%', 'a') ('0.00%', 'lar

Case #6:
Original sentence:
great food and great environment, only snag i ve experienced here is the wait time previous visit took about 45 minutes for our pizza to come out and this most recent time they put the wrong toppings on it so we had to wait for a whole other pizza to be made, regardless of that though it s a good place to have a nice dimlit dinner with reasonably priced bottles of wine, all pizzas are thin crust which i m usually not a fan of but was happy to indulge in our pie during and after the meal

Exact label: 4, predict label: 4

Sentence attention:
100.00% great food and great environment
0.00% only snag i ve experienced here is the wait time previous visit took about 45 minutes for our pizza to come out and this most recent time they put the wrong toppings on it so we had to wait for a whole other pizza to be made
0.00% regardless of that though it s a good place to have a nice <UNKNOWN> dinner with reasonably priced bottles of wine
0.00% all pizzas are thin crust 